# ETS Model Benchmark (Automatic Selection Only)

Parallel evaluation of ETS implementations with automatic component selection on M1, M3, and Tourism competition datasets.

**Packages evaluated:**
- **smooth** (ADAM, ES) - Automatic selection via Z (AIC-based)
- **statsforecast** - AutoETS (Nixtla)
- **sktime** - AutoETS
- **aeon** - ETS with AIC-based automatic selection

**Metrics:**
- RMSSE (Root Mean Squared Scaled Error)
- SAME (Scaled Absolute Mean Error)
- Computational time

In [8]:
import numpy as np
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

from fcompdata import M1, M3, Tourism

In [9]:
# Check which packages are available
available_packages = {}

try:
    from smooth import ADAM, ES
    available_packages['smooth'] = True
    print("smooth: Available")
except ImportError as e:
    available_packages['smooth'] = False
    print(f"smooth: Not available ({e})")

try:
    from statsforecast import models as sf_models
    available_packages['statsforecast'] = True
    print("statsforecast: Available")
except ImportError as e:
    available_packages['statsforecast'] = False
    print(f"statsforecast: Not available ({e})")

try:
    from sktime.forecasting import ets as sktime_ets
    available_packages['sktime'] = True
    print("sktime: Available")
except ImportError as e:
    available_packages['sktime'] = False
    print(f"sktime: Not available ({e})")

try:
    from skforecast.stats import Ets
    available_packages['skforecast'] = True
    print("skforecast: Available")
except ImportError as e:
    available_packages['skforecast'] = False
    print(f"skforecast: Not available ({e})")

try:
    from aeon.forecasting.stats import AutoETS as AeonAutoETS
    available_packages['aeon'] = True
    print("aeon: Available")
except ImportError as e:
    available_packages['aeon'] = False
    print(f"aeon: Not available ({e})")

smooth: Available
statsforecast: Available
sktime: Available
skforecast: Available
aeon: Available


## Error Metrics

In [10]:
def RMSSE(holdout, forecast, actuals):
    """
    Root Mean Squared Scaled Error.
    """
    holdout = np.asarray(holdout)
    forecast = np.asarray(forecast)
    actuals = np.asarray(actuals)
    
    mse = np.mean((holdout - forecast) ** 2)
    scale = np.mean(np.diff(actuals) ** 2)
    
    if scale == 0:
        return np.nan
    
    return np.sqrt(mse / scale)

def SAME(holdout, forecast, actuals):
    """
    Scaled Absolute Mean Error.
    """
    holdout = np.asarray(holdout)
    forecast = np.asarray(forecast)
    actuals = np.asarray(actuals)
    
    ame = np.abs(np.mean(holdout - forecast))
    scale = np.mean(np.abs(np.diff(actuals)))
    
    if scale == 0:
        return np.nan
    
    return ame / scale

## Load Datasets

In [11]:
# Combine datasets into a list
datasets = []
for idx in M1.keys():
    datasets.append(M1[idx])
for idx in M3.keys():
    datasets.append(M3[idx])
for idx in Tourism.keys():
    datasets.append(Tourism[idx])

print(f"Total series: {len(datasets)}")
print(f"M1: {len(M1)} series")
print(f"M3: {len(M3)} series")
print(f"Tourism: {len(Tourism)} series")

Total series: 5315
M1: 1001 series
M3: 3003 series
Tourism: 1311 series


## Define Methods

Methods are defined as tuples: `(method_name, package, config_dict)`

In [18]:
# All method configurations (automatic selection only)
all_methods_config = [
    # smooth package - ADAM and ES models with automatic selection (Z = AIC-based)
    ("ADAM ETS Back", "smooth", {"class": "ADAM", "model": "ZXZ", "initial": "backcasting"}),
    ("ADAM ETS Opt", "smooth", {"class": "ADAM", "model": "ZXZ", "initial": "optimal"}),
    ("ADAM ETS Two", "smooth", {"class": "ADAM", "model": "ZXZ", "initial": "two-stage"}),
    ("ES Back", "smooth", {"class": "ES", "model": "ZXZ", "initial": "backcasting"}),
    ("ES Opt", "smooth", {"class": "ES", "model": "ZXZ", "initial": "optimal"}),
    ("ES Two", "smooth", {"class": "ES", "model": "ZXZ", "initial": "two-stage"}),
    ("ES XXX", "smooth", {"class": "ES", "model": "XXX", "initial": "backcasting"}),
    ("ES ZZZ", "smooth", {"class": "ES", "model": "ZZZ", "initial": "backcasting"}),
    ("ES FFF", "smooth", {"class": "ES", "model": "FFF", "initial": "backcasting"}),
    ("ES SXS", "smooth", {"class": "ES", "model": "SXS", "initial": "backcasting"}),
    
    # statsforecast - AutoETS (AIC-based selection)
    ("statsforecast AutoETS", "statsforecast", {}),
    
    # sktime - AutoETS (AIC-based selection)
    # ("sktime AutoETS", "sktime", {}),

    # skforecast - Ets with automatic selection (ZZZ)
    ("skforecast AutoETS", "skforecast", {}),

    # aeon - ETS with AIC-based automatic selection over all model combinations
    ("aeon AutoETS", "aeon", {}),
]

# Filter to only available packages
methods_config = [
    (name, pkg, cfg) for name, pkg, cfg in all_methods_config 
    if available_packages.get(pkg, False)
]

methods_names = [m[0] for m in methods_config]
methods_number = len(methods_names)
dataset_length = len(datasets)

print(f"\nMethods available: {methods_number}")
print(f"Datasets: {dataset_length}")
print("\nMethod configurations:")
for name, pkg, cfg in methods_config:
    print(f"  {name} ({pkg})")


Methods available: 13
Datasets: 5315

Method configurations:
  ADAM ETS Back (smooth)
  ADAM ETS Opt (smooth)
  ADAM ETS Two (smooth)
  ES Back (smooth)
  ES Opt (smooth)
  ES Two (smooth)
  ES XXX (smooth)
  ES ZZZ (smooth)
  ES FFF (smooth)
  ES SXS (smooth)
  statsforecast AutoETS (statsforecast)
  skforecast AutoETS (skforecast)
  aeon AutoETS (aeon)


## Evaluation Functions

Each package has a different API, so we need separate evaluation functions.

In [13]:
def _evaluate_task(args):
    """
    Worker function for parallel evaluation.
    Handles different package APIs (automatic selection models only).
    """
    import numpy as np
    import time
    import warnings
    warnings.filterwarnings('ignore')
    
    series_idx, series, method_name, package, config = args
    
    try:
        start_time = time.time()
        forecast_values = None
        
        # ===== SMOOTH PACKAGE =====
        if package == "smooth":
            from smooth import ADAM, ES
            
            model_class = config.get("class", "ES")
            model_str = config.get("model", "ZXZ")
            initial = config.get("initial", "backcasting")
            
            if series.period > 1:
                lags = [1, series.period]
            else:
                lags = [1]
                if len(model_str) == 3 and model_str[2] != 'N':
                    model_str = model_str[:2] + 'N'
            
            ModelClass = ADAM if model_class == "ADAM" else ES
            model = ModelClass(model=model_str, lags=lags, initial=initial)
            model.fit(series.x)
            forecasts = model.predict(h=series.h)
            forecast_values = forecasts['mean'].values
        
        # ===== STATSFORECAST =====
        elif package == "statsforecast":
            from statsforecast import models as sf_models
            
            season_length = series.period if series.period > 1 else 1
            model = sf_models.AutoETS(season_length=season_length)
            model.fit(series.x)
            forecasts = model.predict(h=series.h)
            forecast_values = forecasts['mean'] if isinstance(forecasts, dict) else forecasts
        
        # ===== SKTIME =====
        elif package == "sktime":
            import pandas as pd
            from sktime.forecasting import ets as sktime_ets

            y = pd.Series(series.x)
            fh = np.arange(1, series.h + 1)
            sp = series.period if series.period > 1 else 1
            model = sktime_ets.AutoETS(sp=sp, auto=True)
            model.fit(y)
            forecast_values = model.predict(fh).values

        # ===== SKFORECAST =====
        elif package == "skforecast":
            from skforecast.stats import Ets

            season_length = series.period if series.period > 1 else 1
            model = Ets(model="ZZZ", m=season_length)
            model.fit(series.x)
            forecast_values = model.predict(steps=series.h)

        # ===== AEON =====
        elif package == "aeon":
            from aeon.forecasting.stats import AutoETS as AeonAutoETS

            model = AeonAutoETS()
            model.fit(series.x)
            forecast_values = model.iterative_forecast(series.x, series.h)

        else:
            raise ValueError(f"Unknown package: {package}")
        
        time_elapsed = time.time() - start_time
        
        forecast_values = np.asarray(forecast_values).flatten()[:series.h]
        
        # Calculate metrics
        mse = np.mean((series.xx - forecast_values) ** 2)
        scale = np.mean(np.diff(series.x) ** 2)
        rmsse = np.sqrt(mse / scale) if scale != 0 else np.nan
        
        ame = np.abs(np.mean(series.xx - forecast_values))
        scale_same = np.mean(np.abs(np.diff(series.x)))
        same = ame / scale_same if scale_same != 0 else np.nan
        
        return (series_idx, method_name, rmsse, same, time_elapsed)
    
    except Exception as e:
        return (series_idx, method_name, np.nan, np.nan, np.nan)


def evaluate_parallel(datasets, methods_config, n_workers=None):
    """
    Evaluate all methods on all datasets in parallel.
    """
    if n_workers is None:
        n_workers = multiprocessing.cpu_count()
    
    methods_names = [m[0] for m in methods_config]
    n_methods = len(methods_names)
    n_datasets = len(datasets)
    
    results = np.full((n_methods, n_datasets, 3), np.nan)
    
    tasks = []
    for method_name, package, config in methods_config:
        for i, series in enumerate(datasets):
            tasks.append((i, series, method_name, package, config))
    
    print(f"Starting parallel evaluation with {n_workers} workers...")
    print(f"Total tasks: {len(tasks)} ({n_methods} methods × {n_datasets} series)")
    
    start_time = time.time()
    completed = 0
    
    with ProcessPoolExecutor(max_workers=n_workers) as executor:
        futures = {executor.submit(_evaluate_task, task): task for task in tasks}
        
        for future in as_completed(futures):
            result = future.result()
            series_idx, method_name, rmsse, same, elapsed = result
            
            method_idx = methods_names.index(method_name)
            results[method_idx, series_idx, 0] = rmsse
            results[method_idx, series_idx, 1] = same
            results[method_idx, series_idx, 2] = elapsed
            
            completed += 1
            if completed % 1000 == 0:
                elapsed_total = time.time() - start_time
                rate = completed / elapsed_total
                remaining = (len(tasks) - completed) / rate
                print(f"  Progress: {completed}/{len(tasks)} ({100*completed/len(tasks):.1f}%) - "
                      f"ETA: {remaining/60:.1f} min")
    
    total_time = time.time() - start_time
    print(f"\nCompleted in {total_time/60:.1f} minutes ({total_time:.1f}s)")
    print(f"Average time per task: {total_time/len(tasks)*1000:.1f}ms")
    
    return results

## Run Evaluation

In [19]:
print(f"Available CPU cores: {multiprocessing.cpu_count()}")

# Run parallel evaluation
test_results = evaluate_parallel(datasets, methods_config)

# Print summary
print("\nPer-method summary:")
for j, method in enumerate(methods_names):
    rmsse_mean = np.nanmean(test_results[j, :, 0])
    same_mean = np.nanmean(test_results[j, :, 1])
    time_mean = np.nanmean(test_results[j, :, 2])
    failed = np.sum(np.isnan(test_results[j, :, 0]))
    print(f"  {method:24s}: RMSSE={rmsse_mean:.4f}, SAME={same_mean:.4f}, "
          f"Time={time_mean:.3f}s, Failed={failed}")

# Save results
import datetime
date_str = datetime.datetime.now().strftime("%Y-%m-%d")
np.save(f'{date_str}-benchmark-ets-all-no-sktime.npy', test_results)

Available CPU cores: 32
Starting parallel evaluation with 32 workers...
Total tasks: 69095 (13 methods × 5315 series)
  Progress: 1000/69095 (1.4%) - ETA: 7.7 min
  Progress: 2000/69095 (2.9%) - ETA: 5.6 min
  Progress: 3000/69095 (4.3%) - ETA: 5.8 min
  Progress: 4000/69095 (5.8%) - ETA: 5.9 min
  Progress: 5000/69095 (7.2%) - ETA: 6.6 min
  Progress: 6000/69095 (8.7%) - ETA: 7.5 min
  Progress: 7000/69095 (10.1%) - ETA: 7.9 min
  Progress: 8000/69095 (11.6%) - ETA: 8.1 min
  Progress: 9000/69095 (13.0%) - ETA: 9.8 min
  Progress: 10000/69095 (14.5%) - ETA: 11.7 min
  Progress: 11000/69095 (15.9%) - ETA: 11.2 min
  Progress: 12000/69095 (17.4%) - ETA: 11.8 min
  Progress: 13000/69095 (18.8%) - ETA: 11.6 min
  Progress: 14000/69095 (20.3%) - ETA: 12.4 min
  Progress: 15000/69095 (21.7%) - ETA: 13.7 min
  Progress: 16000/69095 (23.2%) - ETA: 13.4 min
  Progress: 17000/69095 (24.6%) - ETA: 12.6 min
  Progress: 18000/69095 (26.1%) - ETA: 11.9 min
  Progress: 19000/69095 (27.5%) - ETA: 11.

## Results Summary

In [ ]:
test_results = np.load('2026-02-27-benchmark-ets-all-no-sktime.npy')

# Create RMSSE summary DataFrame (all series)
summary_rmsse = pd.DataFrame({
    'Method': methods_names,
    'Min': [np.nanmin(test_results[j, :, 0]) for j in range(methods_number)],
    'Q1': [np.nanquantile(test_results[j, :, 0], 0.25) for j in range(methods_number)],
    'Med': [np.nanmedian(test_results[j, :, 0]) for j in range(methods_number)],
    'Q3': [np.nanquantile(test_results[j, :, 0], 0.75) for j in range(methods_number)],
    'Max': [np.nanmax(test_results[j, :, 0]) for j in range(methods_number)],
    'Mean': [np.nanmean(test_results[j, :, 0]) for j in range(methods_number)],
    'Mean Time (s)': [np.nanmean(test_results[j, :, 2]) for j in range(methods_number)],
    'Failed': [np.sum(np.isnan(test_results[j, :, 0])) for j in range(methods_number)]
})

print("\n" + "="*85)
print("EVALUATION RESULTS for RMSSE (All Series)")
print("="*85)
print(summary_rmsse.to_string(index=False))


EVALUATION RESULTS for RMSSE (All Series)
               Method      Min       Q1      Med       Q3          Max         Mean
        ADAM ETS Back 0.018252 0.663358 1.161473 2.301861 5.025854e+01     1.928086
         ADAM ETS Opt 0.024155 0.670682 1.185932 2.365498 5.161599e+01     1.943729
         ADAM ETS Two 0.024599 0.669522 1.182516 2.342385 5.161599e+01     1.947715
              ES Back 0.018252 0.667225 1.160971 2.313932 5.025854e+01     1.927436
               ES Opt 0.024155 0.673575 1.185756 2.364915 5.161599e+01     1.947180
               ES Two 0.024467 0.671771 1.187368 2.346343 5.161599e+01     1.955076
               ES XXX 0.018252 0.677717 1.170823 2.306197 5.025854e+01     1.961318
               ES ZZZ 0.011386 0.670211 1.179916 2.353334 1.155442e+02     2.053459
               ES FFF 0.011386 0.680956 1.211736 2.449626 1.155442e+02     2.100899
               ES SXS 0.018252 0.674537 1.169187 2.353334 5.025854e+01     1.939847
statsforecast AutoETS 0.024468 0.

In [ ]:
# Create SAME summary DataFrame (all series)
summary_same = pd.DataFrame({
    'Method': methods_names,
    'Min': [np.nanmin(test_results[j, :, 1]) for j in range(methods_number)],
    'Q1': [np.nanquantile(test_results[j, :, 1], 0.25) for j in range(methods_number)],
    'Med': [np.nanmedian(test_results[j, :, 1]) for j in range(methods_number)],
    'Q3': [np.nanquantile(test_results[j, :, 1], 0.75) for j in range(methods_number)],
    'Max': [np.nanmax(test_results[j, :, 1]) for j in range(methods_number)],
    'Mean': [np.nanmean(test_results[j, :, 1]) for j in range(methods_number)],
    'Mean Time (s)': [np.nanmean(test_results[j, :, 2]) for j in range(methods_number)],
    'Failed': [np.sum(np.isnan(test_results[j, :, 1])) for j in range(methods_number)]
})

print("\n" + "="*85)
print("EVALUATION RESULTS for SAME (All Series)")
print("="*85)
print(summary_same.to_string(index=False))


EVALUATION RESULTS for SAME (All Series)
               Method      Min       Q1      Med       Q3          Max         Mean
        ADAM ETS Back 0.001142 0.374466 0.995272 2.402342 5.234177e+01     1.951070
         ADAM ETS Opt 0.000106 0.373551 1.021661 2.485596 5.510179e+01     1.962040
         ADAM ETS Two 0.000782 0.380398 1.029629 2.451008 5.510186e+01     1.970422
              ES Back 0.001142 0.372777 0.994547 2.412503 5.345041e+01     1.946517
               ES Opt 0.000217 0.372725 1.024666 2.478435 5.468603e+01     1.967217
               ES Two 0.000095 0.384795 1.028561 2.454543 5.468558e+01     1.982261
               ES XXX 0.000094 0.373315 1.005006 2.425682 5.316973e+01     1.992656
               ES ZZZ 0.000760 0.386673 1.017732 2.467912 1.457604e+02     2.107522
               ES FFF 0.000597 0.401426 1.048395 2.566151 1.457604e+02     2.173559
               ES SXS 0.000597 0.375438 1.005603 2.450490 5.345041e+01     1.964322
statsforecast AutoETS 0.000228 0.3

In [ ]:
# Create Time summary DataFrame (all series)
summary_time = pd.DataFrame({
    'Method': methods_names,
    'Min': [np.nanmin(test_results[j, :, 2]) for j in range(methods_number)],
    'Q1': [np.nanquantile(test_results[j, :, 2], 0.25) for j in range(methods_number)],
    'Med': [np.nanmedian(test_results[j, :, 2]) for j in range(methods_number)],
    'Q3': [np.nanquantile(test_results[j, :, 2], 0.75) for j in range(methods_number)],
    'Max': [np.nanmax(test_results[j, :, 2]) for j in range(methods_number)],
    'Mean': [np.nanmean(test_results[j, :, 2]) for j in range(methods_number)],
    'Total (min)': [np.nansum(test_results[j, :, 2]) / 60 for j in range(methods_number)],
    'Failed': [np.sum(np.isnan(test_results[j, :, 2])) for j in range(methods_number)]
})

print("\n" + "="*85)
print("EVALUATION RESULTS for Computational Time in seconds (All Series)")
print("="*85)
print(summary_time.to_string(index=False))


EVALUATION RESULTS for Computational Time in seconds (All Series)
               Method      Min       Q1      Med       Q3      Max     Mean
        ADAM ETS Back 0.008679 0.086219 0.140929 0.241768 0.923601 0.181546
         ADAM ETS Opt 0.051302 0.229400 0.315379 0.792827 2.638193 0.550378
         ADAM ETS Two 0.051314 0.287382 0.455149 1.080276 3.535120 0.715090
              ES Back 0.009274 0.085299 0.139511 0.247114 0.958868 0.182547
               ES Opt 0.053176 0.224293 0.312200 0.772161 2.888662 0.541243
               ES Two 0.048539 0.279598 0.446404 1.058847 3.553156 0.703183
               ES XXX 0.008793 0.054139 0.093792 0.144012 0.480976 0.104800
               ES ZZZ 0.010502 0.127297 0.184561 0.447649 1.794031 0.313157
               ES FFF 0.046921 0.215119 1.110657 1.557724 3.489375 1.071004
               ES SXS 0.024909 0.116427 0.434594 0.564973 1.251257 0.403988
statsforecast AutoETS 0.001807 0.007699 0.077681 0.156596 1.228376 0.097996
   skforecast AutoETS

## Results without sktime and darts

In [16]:

# Create RMSSE summary DataFrame (all series)
summary_rmsse_subset = summary_rmsse.iloc[:-2]

print("\n" + "="*85)
print("EVALUATION RESULTS for RMSSE (All Series)")
print("="*85)
print(summary_rmsse_subset.to_string(index=False))


EVALUATION RESULTS for RMSSE (All Series)
               Method      Min       Q1      Med       Q3          Max          Mean  Mean Time (s)  Failed
        ADAM ETS Back 0.018252 0.690742 1.213972 2.369759 5.025854e+01      1.985268       0.256999       1
         ADAM ETS Opt 0.024155 0.685189 1.215757 2.419989 5.161599e+01      1.973914       0.606002       0
         ADAM ETS Two 0.024599 0.687406 1.216693 2.400761 5.161599e+01      1.988111       0.768236       0
              ES Back 0.018252 0.696061 1.223922 2.364590 5.025854e+01      1.984016       0.257924       6
               ES Opt 0.024155 0.687564 1.212298 2.410473 5.161599e+01      1.974005       0.590417       1
               ES Two 0.024467 0.690250 1.209945 2.426384 5.161599e+01      1.997188       0.756772       4
               ES XXX 0.018252 0.677717 1.170823 2.306197 5.025854e+01      1.961318       0.103819       0
               ES ZZZ 0.011685 0.704193 1.260036 2.488038 1.836171e+02      2.199855       0.

In [17]:
# Build series ID list matching the order in `datasets`
series_ids = list(M1.keys()) + list(M3.keys()) + list(Tourism.keys())
dataset_sizes = {"M1": len(M1), "M3": len(M3), "Tourism": len(Tourism)}
dataset_boundaries = {}
offset = 0
for name, size in dataset_sizes.items():
    dataset_boundaries[name] = (offset, offset + size)
    offset += size

print("=" * 70)
print("FAILED SERIES DIAGNOSTICS")
print("=" * 70)

any_failures = False
for j, method in enumerate(methods_names):
    failed_mask = np.isnan(test_results[j, :, 0])
    n_failed = failed_mask.sum()
    if n_failed == 0:
        continue
    any_failures = True
    failed_indices = np.where(failed_mask)[0]

    print(f"\n{method} ({n_failed} failures):")
    for ds_name, (lo, hi) in dataset_boundaries.items():
        ds_failed = [series_ids[i] for i in failed_indices if lo <= i < hi]
        if ds_failed:
            print(f"  {ds_name}: {', '.join(str(s) for s in ds_failed)}")

if not any_failures:
    print("\nNo failures detected.")

FAILED SERIES DIAGNOSTICS

ADAM ETS Back (1 failures):
  M1: 653

ES Back (6 failures):
  M1: 182, 653, 870, 945
  M3: 1422
  Tourism: 100

ES Opt (1 failures):
  Tourism: 100

ES Two (4 failures):
  M1: 182
  M3: 2093
  Tourism: 100, 420

ES ZZZ (8 failures):
  M1: 182, 945
  M3: 1422
  Tourism: 100, 1032, 1156, 1170, 1173

ES FFF (14 failures):
  M1: 222, 942, 945
  M3: 351, 1422, 1690
  Tourism: 100, 1032, 1156, 1165, 1166, 1170, 1173, 1278

ES SXS (3 failures):
  M1: 222, 945
  Tourism: 100

aeon AutoETS (2 failures):
  M1: 83
  M3: 118
